In [ ]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

# Formulating and Solving ILPs using PULP.

We have already encountered PuLP as a package for solving linear programs. As it turns out, PuLP is quite useful in solving integer linear programming problems as well. We will show a few examples as an introduction to formulating and solving integer linear programming problems using PuLP. The overall approach is very similar to how linear programming problems are setup. We simply need to make sure that the variables are declared as integer or binary variables.

Let us start with a simple example.

## Example 1 : Variant of Knapsack Problem

<div class="alert alert-block" style="background-color:lightgray; border-color:black white black white">
You are at a candy store and would like to buy $100$ grams of candies. We have the problem of choosing candies to buy (think of this as the ILP version of the diet problem). We have a bunch of $n$ different varieties of candies $D_1, \ldots, D_n$ that we could choose from.
<ul>
  <li> For each candy $D_j$, we can choose at most $k_j$ candies.
  <li> The cost of one piece of candy type $D_j$ is given by $p_j$.
  <li> The number of Calories/piece in candy type $D_j$ is given by $c_j$.
  <li> We need to minimize the overall cost of our purchase.
  <li> A candy gift box can hold at most $N$ candies and we wish our candies to fit inside a gift box.
  <li> The number of calories must be at least $C_{\min}$ and at most $C_{\max}$.
</ul>
    
 The problem is an integer linear program since we have to choose a whole number of candies. The problem data is given by $n$ the number of candy types, $(k_1, \ldots, k_n)$ how many of each candy type are available for purchase, the prices $(p_1, \ldots, p_n)$, the calories/piece $(c_1, \ldots, c_n)$, the limit $N$ on number of candies per box and caloric limits $C_{\min}$ and $C_{\max}$.
    
The ILP has decision variables
    $$x_1, \ldots, x_n,$$
wherein $x_i$ denotes the number of candies of type $i$ that are to be purchased. Each $x_i \in \mathbb{Z}$ (is an integer) and must satisfy the limits:
    $$ 0 \leq x_i \leq k_i,\ i = 1, \ldots, n $$
Next, the number of candies chosen cannot exceed $N$:
    $$ \sum_{i=1}^n x_i \leq N $$
We must respect the caloric limits:
    $$ C_{\min} \leq \underset{\text{Calories Consumed}}{\underbrace{\sum_{i=1}^n c_i x_i}}  \leq C_{\max} $$

Finally, the objective is to minimize cost. The overall ILP is

$$\begin{array}{rll}
\min& \sum_{j=1}^n p_j x_j & \leftarrow\ \text{minimize total cost of purchase} \\ 
\mathsf{s.t.} & 0 \leq x_j \leq k_j & \leftarrow\ \text{limit on max. number of candies of each type} \\ 
& \sum_{j=1}^n x_j \leq N & \leftarrow\ \text{limit on total number of candies/box}\\ 
& C_{\min} \leq \sum_{i=1}^n c_i x_i  \leq C_{\max}& \leftarrow\ \text{calorie limits} \\ 
& x_1, \ldots, x_n \in \mathbb{Z} & \leftarrow\ \text{integrality constraints}\\
\end{array}$$
</div>

Let's implement this model in python using pulp

In [1]:
from pulp import * # get all the definitions from pulp library

def solve_candy_knapsack(n, candy_number_limits, candy_prices, candy_calories, N, Cmin, Cmax):
    assert len(candy_number_limits) == n, 'size mismatch'
    assert len(candy_prices) == n, 'size mismatch for prices'
    assert len(candy_calories) == n, 'size mismatch for list of calories'
    assert N >= 1, 'total number of candies per box must be 1 or more'
    assert Cmin <= Cmax, 'minimum calories is greater than the maximum calories'
    prob = LpProblem('Candy Knapsack', LpMinimize)
    # add decision variables
    # make a list of n decision variables, one for each candy. When declaring the variable, automatically declare
    # its lower bound to be 0 and upper bound to be ki from the candy_number_limits array
    # also declare the category (cat) of the variable to be integers.
    dVars = [LpVariable(f'x{i}',lowBound=0, upBound=ki, cat='Integer') for (i, ki) in enumerate(candy_number_limits)]
    # Now set the objective
    prob += lpSum([pj*xj for (pj,xj) in zip(candy_prices, dVars)])
    # Now add the constraints
    prob += lpSum(dVars) <= N # constraints on number of candies per box
    calories_of_candies = lpSum([cj*xj for (cj,xj) in zip(candy_calories, dVars)])
    prob += calories_of_candies <= Cmax
    prob += calories_of_candies >= Cmin
    status = prob.solve()
    if status == constants.LpStatusInfeasible:
        print('Problem is infeasible')
        return
    elif status == constants.LpStatusUnbounded:
        print('Problem is unbounded. Cannot proceed')
        return
    else:
        assert status == constants.LpStatusOptimal, 'Something went wrong while solving since status is either undefined or unsolved'
        # extract values
        print('Success: optimal answer found')
        solution_values = [x.varValue for x in dVars]
        for (j, svj) in enumerate(solution_values):
            print(f'\t Candy Type # {j}: {svj} candies')
        print(f'Total Cost: {sum([(pj*svj) for (pj, svj) in zip(candy_prices, solution_values)])}')
        print(f'Calories: {sum([cj*xj for (cj,xj) in zip(candy_calories, solution_values)])}')

In [2]:
n = 5
candy_number_limits = [10, 12, 10, 11, 10]
candy_prices = [0.2, 0.5, 0.1, 0.4, 0.8]
candy_calories = [25, 12, 22, 14, 33]
solve_candy_knapsack(5, candy_number_limits, candy_prices, candy_calories, 12, 250, 500)


Success: optimal answer found
	 Candy Type # 0: 2.0 candies
	 Candy Type # 1: 0.0 candies
	 Candy Type # 2: 10.0 candies
	 Candy Type # 3: 0.0 candies
	 Candy Type # 4: 0.0 candies
Total Cost: 1.4
Calories: 270.0


c:\Python312\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Set Cover Problem


We will now present another example involving binary variables. 

<div class="alert alert-block" style="background-color:lightgray; border-color:black white black white">

Let's assume that you are planning a company party and there are $n$ people conveniently numbered $1, \ldots, n$ that you could invite. We wish to invite at least $m$ people to attend. Therefore, HR tells you to respect the following constraints:
<ul>
    <li> There are teams of employees $T_1, \ldots, T_k$ wherein $T_i \subseteq \{1, \ldots, n\}$ is a subset of employees. An employee can be in multiple teams. We wish to ensure that at least $1/4$th of the members from each team are invited. Note that the same person could be invited as a representative of multiple teams.
    <li> We are given a set of pairs $G_1, \ldots, G_l$ wherein each $G_j$ consists of a set of employees who are involved in grievances against each other. We cannot invite more than one employee from each grievance set $G_j$ in order to keep the party festive.
    <li> The HR department with the help of its machine learning algorithms has assigned a <i> party pooper</i> score $s_i$ to employee $i$ where the higher the score, the more likely they are to be a party pooper. We wish to minimize the total party pooper score of all the employees invited to the party.
</ul>
    
<b> Decision Variables: </b> We will have decision variables $w_1, \ldots, w_n$ wherein $w_i = 1 $ indicates that employee $i$ is invited to the party and $w_i = 0$ indicates that they are not.

<b> Objective : </b>  $\min\ \sum_{i=1}^n s_i w_i$ wherein $s_i$ is the "party pooper" score for employee $i$.
   
<b> Constraints: </b> We will have the following constraints:
<ul>
    <li> $ \sum_{j \in T_i} w_j \geq |T_i|/4\ \text{for each}\ T_i$ (at least one fourth of members from each team)
    <li> $ \sum_{j \in G_i} w_j \leq 1\ \text{for each}\ G_i$ (at most one from each grievance set)
    <li> $ \sum_{j=1}^n w_j \geq m$ (at least $m$ people invited in total )
    <li> $ w_j \in \{0, 1\}$ for all $j \in \{1, \ldots, n\}$ (binary variables)
</ul>
</div>

In [4]:
def plan_invite_list(n, m, T_lists, G_lists, pp_scores):
    assert m >= 0, 'Cannot invite a negative number of people'
    assert all( 0 <= j < n for ti in T_lists for j in ti ) # check that all employee IDs are valid
    assert all( 0<= j < n for gi in G_lists for j in gi)
    assert len(pp_scores) == n, 'Length of party pooper scores list does not match number of employees'
    prob = LpProblem('PartyPlanner', LpMinimize)
    # create decision variables
    dvars = [LpVariable(f'w_{i}', cat='Binary') for i in range(n)] # declare variables as binary
    # if we declared variables as binary they are automatically treated as either 0 or 1 in the optimization
    # create objective
    prob += lpSum([si * wi for (si, wi) in zip(pp_scores, dvars)])
    # limit on number of invitees
    prob += sum(dvars) >= m
    # at least two people from each team
    for ti in T_lists:
        prob += lpSum([dvars[j] for j in ti]) >= len(ti)/4
    # no more than one person per grievance set
    for gj in G_lists:
        prob += lpSum(dvars[j] for j in gj) <= 1
    # solve and get the result
    status = prob.solve()
    if status == constants.LpStatusInfeasible:
        print('infeasible LP')
        return
    elif status != constants.LpStatusOptimal:
        print('Unbounded or undefined LP Status -- there is some mistake in the problem formulation since it cannot happen in theory')
        return
    else:
        assert status == constants.LpStatusOptimal
        # extract values
        sol = [x.varValue for x in dvars]
        for (j,wj) in enumerate(sol):
            if wj >= 1:
                print(f'Invite person {j}')
        print(f'Total # of invitees: {sum(sol)}')
        for j,tj in enumerate(T_lists):
            print(f'# of invitees from Team # {j}: {sum([sol[k] for k in tj])}')
        for k, gk in enumerate(G_lists):
            print(f'# of invitees from Grievance set # {k}: {sum([sol[j] for j in gk])}')
        return


In [5]:
n = 20
m = 12
T_lists = [[1, 5, 12, 18, 19], [2, 3, 4, 6, 7], [1, 2, 4, 7, 8, 9, 10, 11, 12, 14, 16], [1, 3, 4, 5, 6, 13, 15, 17, 18, 19], [1, 5, 7, 8,9, 19]]
G_lists = [[1, 5], [5, 19], [4, 7], [4, 12], [4, 19], [4, 18], [3, 4, 15, 19], [4, 7, 18, 2]]
pp_scores = [1, 2, 2, 1, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3.5, 1, 0.6, 0, 1, 8, 8]
plan_invite_list(n, m, T_lists, G_lists, pp_scores)

Invite person 0
Invite person 1
Invite person 2
Invite person 6
Invite person 8
Invite person 11
Invite person 12
Invite person 13
Invite person 14
Invite person 15
Invite person 16
Invite person 17
Total # of invitees: 12.0
# of invitees from Team # 0: 2.0
# of invitees from Team # 1: 2.0
# of invitees from Team # 2: 7.0
# of invitees from Team # 3: 5.0
# of invitees from Team # 4: 2.0
# of invitees from Grievance set # 0: 1.0
# of invitees from Grievance set # 1: 0.0
# of invitees from Grievance set # 2: 0.0
# of invitees from Grievance set # 3: 1.0
# of invitees from Grievance set # 4: 0.0
# of invitees from Grievance set # 5: 0.0
# of invitees from Grievance set # 6: 1.0
# of invitees from Grievance set # 7: 1.0


In [6]:
from random import randint
n = 32
m = 18
num_teams = 30
num_grievances = 6
T_lists = [list(set([randint(0,31) for k in range(randint(3,10))])) for i in range(num_teams)] # 30 random teams
G_lists = [list(set([randint(0,31) for k in range(randint(2,4))])) for i in range(num_grievances)] # 6 random pairs of grievances
for i, ti in enumerate(T_lists):
    print(f'\t Team # {i}: {ti}')
for j, gj in enumerate(G_lists):
    print(f'\t Grievance set # {j}: {gj}')
pp_scores = [randint(0, 8) for i in range(n)]
plan_invite_list(n, m, T_lists, G_lists, pp_scores)

	 Team # 0: [0, 3, 4, 6, 14, 29]
	 Team # 1: [2, 3, 8, 12, 20]
	 Team # 2: [3, 9, 14, 21, 28]
	 Team # 3: [5, 10, 11, 13, 22, 31]
	 Team # 4: [0, 15, 19, 28, 29, 30]
	 Team # 5: [24, 17, 19, 5]
	 Team # 6: [0, 4, 6, 8, 14, 15, 19, 21, 30]
	 Team # 7: [3, 8, 11, 15, 22, 23, 26, 29, 31]
	 Team # 8: [3, 5, 7, 10, 18, 22, 28, 29]
	 Team # 9: [1, 2, 4, 6, 7, 9, 14, 20, 25, 26]
	 Team # 10: [0, 5, 6, 10, 19, 22, 25, 31]
	 Team # 11: [27, 28, 22]
	 Team # 12: [2, 3, 5, 7, 10, 15, 23, 27]
	 Team # 13: [0, 8, 10, 12, 14, 17, 24, 31]
	 Team # 14: [0, 8, 10, 14, 17, 29]
	 Team # 15: [11, 28, 13]
	 Team # 16: [9, 13, 14, 16, 27]
	 Team # 17: [25, 11, 14]
	 Team # 18: [12, 28, 22]
	 Team # 19: [7, 14, 17, 18, 22, 26]
	 Team # 20: [1, 5, 6, 10, 13, 22, 23]
	 Team # 21: [0, 7, 8, 10, 13, 14, 19, 21, 26, 28]
	 Team # 22: [0, 1, 12, 13, 20, 26, 28]
	 Team # 23: [8, 3, 6]
	 Team # 24: [0, 7, 15, 17, 18]
	 Team # 25: [7, 8, 9, 17, 21, 24]
	 Team # 26: [0, 6, 10, 16, 17, 24, 31]
	 Team # 27: [1, 3, 4, 6, 

# That's All Folks